# Case Study of Whitelee Windfarm
This file is for exploring the case study. It is originally placed at the /root folder.

## Land cover types at current wind turbine locations

In [ ]:
import numpy as np
import netCDF4
from Land.main import land
from CRS.crs_init import CRSConvertor

In [3]:
# read wind turbine locations
turbine_loc = np.loadtxt('Aborted/case/Turbines_At_Whitelee_Wind_Farm.csv', delimiter=',', encoding='utf-8', usecols=[1, 2])
conv = CRSConvertor([55.714704134580245, -4.364543821199962, 55.634359319706036, -4.183104393719847])
print(turbine_loc)

[[55.6891561 -4.3580354]
 [55.6866291 -4.3518556]
 [55.6952387 -4.3633451]
 [55.6934182 -4.3555815]
 [55.6859453 -4.3414069]
 [55.6896599 -4.3292643]
 [55.6964595 -4.3525545]
 [55.7028635 -4.3566589]
 [55.7045988 -4.351543 ]
 [55.7028327 -4.3463191]
 [55.7004356 -4.3407223]
 [55.6941866 -4.3231996]
 [55.6913359 -4.3185243]
 [55.6890699 -4.3130225]
 [55.687036  -4.3079828]
 [55.6840153 -4.3021339]
 [55.6811411 -4.2990084]
 [55.6788471 -4.2912424]
 [55.6802265 -4.284701 ]
 [55.6621995 -4.2430467]
 [55.6572329 -4.2390082]
 [55.655983  -4.23264  ]
 [55.6552007 -4.2252601]
 [55.7072954 -4.3467533]
 [55.7052446 -4.3407187]
 [55.7030703 -4.3359135]
 [55.6976605 -4.3226198]
 [55.6959594 -4.3175033]
 [55.693707  -4.3109932]
 [55.6916351 -4.3072165]
 [55.6892417 -4.3020673]
 [55.686783  -4.2961061]
 [55.6824166 -4.2928225]
 [55.6834495 -4.2855856]
 [55.6811579 -4.278979 ]
 [55.6715497 -4.2544861]
 [55.6661212 -4.2424683]
 [55.6609738 -4.235968 ]
 [55.6599369 -4.2287444]
 [55.709604  -4.3405915]


In [4]:
with netCDF4.Dataset('Land/raw/C3S-LC-L4-LCCS-Map-300m-P1Y-2022-v2.1.1.nc', 'r') as file:
    land = file.variables['lccs_class']
    lat = file.variables['lat']
    lon = file.variables['lon']

    # a list to store land cover classes occurred in current layout
    occur = []

    for i in range(turbine_loc.shape[0]):
        dist = np.abs(lat[:] - turbine_loc[i, 0])
        iy_min = dist.argmin()
        dist = np.abs(lon[:] - turbine_loc[i, 1])
        ix_min = dist.argmin()
        temp = land[0, iy_min, ix_min].data
        if temp not in occur:
            occur.append(int(temp))
        print(temp)
    
    print(occur)

180
180
180
180
70
180
180
180
180
180
180
180
180
110
180
70
110
110
70
100
70
110
110
180
180
180
210
180
180
70
70
70
70
180
180
110
100
70
70
180
180
180
130
180
180
180
70
180
180
100
70
100
180
180
130
130
180
180
180
110
100
180
180
100
70
100
100
130
130
130
130
180
180
110
70
110
110
180
100
70
70
70
100
70
70
110
180
70
70
110
180
180
70
70
70
70
110
180
110
100
70
180
100
130
100
70
70
70
100
100
110
70
70
100
70
70
70
70
100
70
70
70
70
70
70
70
70
70
70
70
70
70
70
70
100
100
70
100
70
70
70
70
70
70
110
70
70
110
70
70
70
70
70
70
70
70
70
70
70
70
70
110
100
110
70
100
100
70
70
70
70
180
70
70
70
180
180
100
130
70
70
70
100
100
100
100
70
70
110
70
70
70
70
70
100
130
100
70
70
180
180
70
110
70
70
70
100
70
70
110
110
70
70
70
70
[180, 70, 110, 100, 210, 130]


So, all the current wind turbines are installed in 6 main land cover categorises. They are:

70 Tree cover, needleleaved, evergreen, closed to open

100 Mosaic tree and shrub (>50%) / herbaceous cover (<50%) 

110 Mosaic herbaceous cover (>50%) / tree and shrub (<50%)

130 Grassland

180 Shrub or herbaceous cover, flooded, fresh/saline/brackish water

210 Water bodies.

These categories should be treated as feasible areas. Only 1 turbine is at the location of water bodies (210) due to the resolution issue, so 210 remains infeasible. Grassland (130) has already been feasible. So, Categories 70, 100, 110, and 180 should be changed from infeasible area to feasible area. Besides, categories which are similar to them should also be changed to feasible.

Therefore, change the following categories from infeasible to feasible:
40, 50, 60, 70, 80, 90, 100, 140, 160, 170, 180

## Get land cover classification
This part is for Yiyang to get the data.

In [5]:
def buffer(origin, lower, upper):
    """
    This function is to extend one element at the two ends of a list
    `origin`: an increasingly sorted numpy ndarray of inegers including the original data.
    `lower`: an integer of the lower bounds of the possible values.
    `upper`: an integer of the upper bounds of the possible values. 
    """
    origin = origin.tolist()
    if origin[0] > lower:
        origin.insert(0, origin[0] - 1)
    if origin[-1] < upper:
        origin.append(origin[-1] + 1)
    origin = np.array(origin, dtype='int32')
    return origin


area = [55.714704134580245, -4.364543821199962, 55.634359319706036, -4.183104393719847]
# This is the raw data of land cover classification
with netCDF4.Dataset('Land/raw/C3S-LC-L4-LCCS-Map-300m-P1Y-2022-v2.1.1.nc', 'r') as file:
    land = file.variables['lccs_class']
    lat = file.variables['lat']
    lon = file.variables['lon']
    
    y_range = np.argwhere((lat[:] < area[0]) & (lat[:] > area[2])).T[0]
    x_range = np.argwhere((lon[:] < area[3]) & (lon[:] > area[1])).T[0]

    y_range = buffer(y_range, 0, lat.shape[0])
    x_range = buffer(x_range, 0, lon.shape[0])

    # flip the latitude
    y_range = np.flip(y_range, axis=0)

    print('The latitude grid is:\n', lat[y_range])
    print('The longitude grid is:\n', lon[x_range])

    land_cover = land[0, y_range, x_range].data
    print('The land cover classification is:\n', land_cover)


The latitude grid is:
 [55.63194444 55.63472222 55.6375     55.64027778 55.64305556 55.64583333
 55.64861111 55.65138889 55.65416667 55.65694444 55.65972222 55.6625
 55.66527778 55.66805556 55.67083333 55.67361111 55.67638889 55.67916667
 55.68194444 55.68472222 55.6875     55.69027778 55.69305556 55.69583333
 55.69861111 55.70138889 55.70416667 55.70694444 55.70972222 55.7125
 55.71527778]
The longitude grid is:
 [-4.36527778 -4.3625     -4.35972222 -4.35694444 -4.35416667 -4.35138889
 -4.34861111 -4.34583333 -4.34305556 -4.34027778 -4.3375     -4.33472222
 -4.33194444 -4.32916667 -4.32638889 -4.32361111 -4.32083333 -4.31805556
 -4.31527778 -4.3125     -4.30972222 -4.30694444 -4.30416667 -4.30138889
 -4.29861111 -4.29583333 -4.29305556 -4.29027778 -4.2875     -4.28472222
 -4.28194444 -4.27916667 -4.27638889 -4.27361111 -4.27083333 -4.26805556
 -4.26527778 -4.2625     -4.25972222 -4.25694444 -4.25416667 -4.25138889
 -4.24861111 -4.24583333 -4.24305556 -4.24027778 -4.2375     -4.2347222

In [6]:
# This is the feasibility data
with netCDF4.Dataset('Land/data/infeasible.nc', 'r') as file:
    fea = file.variables['feasible']
    lat = file.variables['latitude']
    lon = file.variables['longitude']
    
    y_range = np.argwhere((lat[:] < area[0]) & (lat[:] > area[2])).T[0]
    x_range = np.argwhere((lon[:] < area[3]) & (lon[:] > area[1])).T[0]

    y_range = buffer(y_range, 0, lat.shape[0])
    x_range = buffer(x_range, 0, lon.shape[0])

    feasibility = np.astype(np.logical_not(fea[y_range, x_range].mask), 'int32')
    print('The feasibility is:\n', feasibility)

The feasibility is:
 [[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]


## Performance of Current Layout

In [11]:
# get the grid cell indices of the current layout
turbine_ind = np.zeros(turbine_loc.shape[0], dtype='int32')
for i in range(turbine_ind.shape[0]):
    dist_sq = np.sum((conv.grid_gcs - turbine_loc[i]) ** 2, axis=1)
    turbine_ind[i] = dist_sq.argmin()  # This is the gene
print(turbine_ind)
print(conv.cols)

[2928 2780 3226 3154 2785 2940 3305 3679 3756 3683 3535 3167 3019 2946
 2798 2651 2502 2355 2433 1475 1176 1104 1107 3908 3835 3687 3393 3320
 3172 3024 2951 2803 2579 2582 2510 1995 1700 1402 1330 4060 3912 3839
 3692 3471 3323 3250 3102 2662 2145 1922 1925 1628 4138 3990 3842 3770
 3622 3475 3327 2739 2515 2443 2371 2148 2076 1853 1781 4140 4142 4069
 3921 3531 3701 3553 2891 2742 2595 2522 2677 2300 2154 2005 1932 2793
 2721 4074 3926 3779 3119 2819 2822 2749 2525 2378 2380 3197 2972 2974
 2902 2604 2532 3124 3127 2979 2829 2573 2685 3350 3277 1555 2982 2910
 2612 3429 3205 3208 3135 2837 2765 2469 2396 3357 3361 3071 2989 2992
 2694 2622 3510 3438 3288 3216 3144 2920 2847 3664 3366 3591 3369 3149
 2201 2129 1904 1680 1833 1685 1763 2057 2208 1532 1384 1537 1311 1088
  935 1158 1231 1379 1527 1301 1153 1081 1075  921  770  927  849  917
  691  543  540  462  392  169  167  239  161   14  384 1593 1521 1298
 1368 1220 1365 1066 1138  914 2194 2048 1976 1823 1752 1599 1969 1817
 1745 

In [12]:
from Optimiser.config import *

_wind_data = np.array([[3.171875, 0.0390625],
                      [4.28125, 0.1015625],
                      [4.1328125, 0.0859375],
                      [3.953125, 0.09375  ],
                      [5.65625, 0.15625  ],
                      [5.6484375, 0.1796875],
                      [5.890625, 0.1171875],
                      [0.,0.]])

# The following part is from Optimiser.main

'''
xy position initialisation
from 1-D index to xy position
'''
rows = conv.rows
cols = conv.cols

xy = np.zeros((rows, cols, 2), dtype='float64')
for i in range(rows):
    xy[i, :, 1] = i
for i in range(cols):
    xy[:, i, 0] = i
xy = xy.reshape(rows * cols, 2)
xy = xy * cell_width + cell_width / 2
xy = xy.transpose()

# trans_matrix is for rotating the coordinates to fit different wind directions.
trans_matrix = np.zeros((len(theta) // 2, 2, 2), dtype='float64')
trans_xy = np.zeros((len(theta) // 2, rows * cols, 2), dtype='float64')
for i in range(len(theta) // 2):
    trans_matrix[i] = np.array(
        [[np.cos(theta[i]), -np.sin(theta[i])],
        [np.sin(theta[i]), np.cos(theta[i])]],
        dtype='float64')
    trans_xy[i] = np.matmul(trans_matrix[i], xy).transpose()

def fitness_func(ga_instance, solution, solution_idx):
    num_genes = ga_instance  # This line is changed
    fitness = 0  # a specific layout power accumulate
    for ind_t in range(len(theta) // 2):
        trans_xy_position = trans_xy[ind_t, solution, :]
        # print(trans_xy_position.shape)

        speed_deficiency0, speed_deficiency1 = wake(trans_xy_position, num_genes)

        # total power of a specific layout under a wind direction
        actual_velocity = (1 - speed_deficiency0) * _wind_data[ind_t, 0]
        lp_power = layout_power(actual_velocity, num_genes)
        fitness += lp_power.sum() * _wind_data[ind_t, 1]
        # calculation for the opposite wind direction
        actual_velocity = (1 - speed_deficiency1) * _wind_data[ind_t + 4, 0]
        lp_power = layout_power(actual_velocity, num_genes)
        fitness += lp_power.sum() * _wind_data[ind_t + 4, 1]
    return fitness


def wake(trans_xy_position, n):
    """
    This function is used by fitness_func().
    """
    # y value increasingly sort
    sorted_index = np.argsort(trans_xy_position[:, 1])
    wake_deficiency0 = np.zeros(n, dtype='float64')
    wake_deficiency1 = np.zeros(n, dtype='float64')
    for j in range(n):
        for k in range(j):
            dx = np.absolute(trans_xy_position[sorted_index[j], 0] - trans_xy_position[sorted_index[k], 0])
            dy = np.absolute(trans_xy_position[sorted_index[j], 1] - trans_xy_position[sorted_index[k], 1])
            d = cal_deficiency(dx=dx, dy=dy)
            # calculate the wake deficiency in two opposite directions at the same time.
            wake_deficiency0[sorted_index[k]] += d ** 2
            wake_deficiency1[sorted_index[j]] += d ** 2
    return np.sqrt(wake_deficiency0), np.sqrt(wake_deficiency1)


def cal_deficiency(dx, dy):
    """
    This function is used by wake().
    """
    r_wake = rotor_radius + entrainment_const * dy
    if dx >= rotor_radius + r_wake:
        intersection = 0
    elif dx > r_wake - rotor_radius:
        alpha = np.arccos((r_wake ** 2 + dx ** 2 - rotor_radius ** 2) / (2 * r_wake * dx))
        beta = np.arccos((rotor_radius ** 2 + dx ** 2 - r_wake ** 2) / (2 * rotor_radius * dx))
        intersection = alpha * r_wake ** 2 + beta * rotor_radius ** 2 - r_wake * dx * np.sin(alpha)
    else:
        intersection = np.pi * rotor_radius ** 2
    return 2.0 / 3.0 * intersection / (np.pi * r_wake ** 2)


def layout_power(v, n):
    """
    This function is used by fitness_func().
    Power unit: kW
    """
    power = np.zeros(n, dtype='float64')
    for j in range(n):
        if 2.0 <= v[j] < 18:
            if v[j] < 12.8:
                power[j] = 0.3 * v[j] ** 3
            else:
                power[j] = 629.1
    return power

print(fitness_func(turbine_ind.shape[0], turbine_ind, None))

4759.976336169799


In [ ]:
# Display the current layout
import folium.features
import folium.raster_layers
import folium
